# Spark Data Acquisition
### Kwame V. Taylor

This exercises uses the ```case.csv```, ```dept.csv```, and ```source.csv``` files from the San Antonio 311 call dataset.

### Exercises

1. Read the case, department, and source data into their own spark dataframes.


2. Let's see how writing to the local disk works in spark:
   * Write the code necessary to store the source data in both csv and json format, store these as ```sources_csv``` and ```sources_json```.
   * Inspect your folder structure. What do you notice?


3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

**Imports**

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

**Acquire data**

In [2]:
source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)
dept = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)
case = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)

**Write data to disk**

In [3]:
source.write.csv("sources_csv", mode="overwrite")

In [4]:
source.write.json("sources_json", mode="overwrite")

I wrote this to the main (spark-exercises/) folder and got rid of the data/ folder. I prefer to have shorter file names over having my data in a folder.

```sources_csv``` is a folder containing these files:
* ```_SUCCESS```
* ```part-00000-10e448c2-f9f9-4610-8f8b-b98d2966fe59-c000.csv```

```sources_json``` is a folder containing these files:
* ```_SUCCESS```
* ```part-00000-2524418d-6636-414c-8962-06b2b3817975-c000.json```

**Inspect schemas and data types**

In [16]:
source.printSchema()
source.show(5)

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



Both fields are strings. This is how I want the data, so I will leave it.

In [17]:
dept.printSchema()
dept.show(5)

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 r

```dept_subject_to_SLA``` needs to be converted into a boolean.

In [20]:
case.printSchema()
case.show(1, vertical=True)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  


```case_opened_date```, ```case_closed_date```, and ```SLA_due_date``` need to be converted into Timestamps.

```case_late``` and ```case_closed``` need to be converted into booleans.

**Clean data - ```dept```**

In [22]:
# convert dept_subject_to_SLA to boolean

dept.groupby('dept_subject_to_SLA').count().show()

+-------------------+-----+
|dept_subject_to_SLA|count|
+-------------------+-----+
|                YES|   31|
|                 NO|    8|
+-------------------+-----+



In [23]:
dept = dept.withColumn('dept_subject_to_SLA', expr("dept_subject_to_SLA == 'YES'"))

In [33]:
dept.show(12)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|               true|
|               Brush|Solid Waste Manag...|           Solid Waste|               true|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|               true|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|               true|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|               true|
|Code Enforcement ...|Code Enforcement ...|  DSD/Code Enforcement|               true|
|Code Enforcement ...|                null|  DSD/Code Enforcement|               true|
|   Dangerous Premise|Code Enforcement ...|  DSD/Code Enforcement|               true|
|Dangerous Premise...|Code Enforcement ...|

**Clean data - ```case```**

In [34]:
# rename SLA_due_date to case_due_date

case = case.withColumnRenamed("SLA_due_date", "case_due_date")

In [35]:
# convert case_late and case_closed to booleans

case.groupby('case_late', 'case_closed').count().show()

+---------+-----------+------+
|case_late|case_closed| count|
+---------+-----------+------+
|       NO|        YES|735616|
|      YES|        YES| 87978|
|       NO|         NO| 11585|
|      YES|         NO|  6525|
+---------+-----------+------+



In [36]:
case = case.withColumn("case_late", expr("case_late == 'YES'"))
case = case.withColumn("case_closed", expr("case_closed == 'YES'"))

In [41]:
case.printSchema()
case.show(2, vertical=True)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- case_due_date: string (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 case_due_date        | 9/26/20 0:42         
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true               

**Convert the dates to timestamps**

In [42]:
fmt = "M/d/yy H:mm"

case.select(
    "case_opened_date",
    to_timestamp("case_opened_date", fmt)
).show(5)

+----------------+-----------------------------------------------+
|case_opened_date|to_timestamp(`case_opened_date`, 'M/d/yy H:mm')|
+----------------+-----------------------------------------------+
|     1/1/18 0:42|                            2018-01-01 00:42:00|
|     1/1/18 0:46|                            2018-01-01 00:46:00|
|     1/1/18 0:48|                            2018-01-01 00:48:00|
|     1/1/18 1:29|                            2018-01-01 01:29:00|
|     1/1/18 1:34|                            2018-01-01 01:34:00|
+----------------+-----------------------------------------------+
only showing top 5 rows



In [43]:
case = case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
case = case.withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
case = case.withColumn("case_due_date", to_timestamp("case_due_date", fmt))

In [44]:
case.printSchema()
case.show(3, vertical=True)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true      

**Clean request_address**

In [45]:
case = case.withColumn("request_address", lower(trim("request_address")))

**Add features**

In [46]:
# Add case_age, days_to_close, and case_lifetime features

case = case.withColumn("case_age", datediff(current_timestamp(), "case_opened_date"))
case = case.withColumn("days_to_close", datediff("case_closed_date", "case_opened_date"))
case = case.withColumn("case_lifetime", when(col("case_closed"), col("days_to_close")).otherwise(col("case_age")))

In [47]:
# Add zip code

case = case.withColumn("zipcode", regexp_extract("request_address", r"(\d+)$", 1))

In [48]:
case.show(3, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 case_age             | 1065                 
 days_to_close        | 0                    
 case_lifetime        | 0                    
 zipcode              | 78207                
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01

### Exercises Continued

1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

2. How many Stray Animal cases are there?

3. How many service requests that are assigned to the Field Operations department (```dept_division```) are not classified as "Officer Standby" request type (```service_request_type```)?

4. Convert the ```council_district``` column to a string column.

5. Extract the year from the ```case_closed_date column```.

6. Convert ```num_days_late``` from days to hours in new columns ```num_hours_late```.

7. Join the case data with the source and department data.

8. Are there any cases that do not have a request source?

9. What are the top 10 service request types in terms of number of requests?

10. What are the top 10 service request types in terms of average days late?

11. Does number of days late depend on department?

12. How do number of days late depend on department and request type?